In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from mrrce import MrRCE
from simulations.simulation_utils import generate_data, model_error
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV
import matplotlib.pyplot as plt
%matplotlib inline

#### Navigate

1. [Generate random dataset](#Generate-random-dataset)
1. [Fit with CV](#Fit-with-CV)
1. [Estimations](#Estimations)
1. [Compare to other estimators](#Compare-to-other-estimators)
1. [Fit with no CV](#Fit-with-no-CV)

##### Generate random dataset

In [3]:
# settings
params = dict(
n = 50,                          # num obs
p = 20,                          # num predictors
q = 5,                           # num tasks
sigma = 1,                       # coeff variance
corr_x = .7,                     # grid of values for rho (correlation coefficient)
sigma_err = 1,                   # correlation coefficient for predictors
err_corr = .9,                   # correlation coefficient for errors
g_sparse_level = .0,             # group sparsity level
sparse_level = .0,               # sparsity level
err_cov_type = 'ar'             # error covariance form. One of ['ar', 'equi', 'fgn', 'identity']
)

In [4]:
np.random.seed(1)
rho = .4 # correlation coefficient for coefficients
X, Y, B, Sigma, Sigma_X = generate_data(rho = rho, **params)

##### Fit with CV

In [5]:
m = MrRCE(
    max_iter=15, 
    tol_glasso=.1, 
    tol=1e-3, 
    glasso_cv=True, 
    n_folds=3, 
    n_lams=10, 
    verbose=True
) # init MrRCE

In [6]:
m.fit(X, Y) # fit MrRCE

INFO:root:iter 1, loss 39.402175
INFO:root:iter 2, loss 35.887994
INFO:root:iter 3, loss 35.228853
INFO:root:iter 4, loss 35.056097
INFO:root:iter 5, loss 35.019021
INFO:root:iter 6, loss 34.994457


##### Estimations

In [7]:
# coefficient matrix
B.round(5)

array([[-0.9364 , -0.82716, -0.31097,  1.10047,  1.52656],
       [ 1.67784,  0.21784, -0.15372,  0.28456,  1.11545],
       [ 0.61821, -0.43367, -2.09498,  0.32065, -0.74634],
       [-0.64194, -0.16023,  1.33785,  1.28063, -0.95908],
       [ 0.72821, -0.27804,  0.15327,  0.54571,  0.13871],
       [-1.39322, -1.60243, -0.23998, -1.90058, -0.01088],
       [ 0.63679, -1.18148, -0.10698, -0.87449,  0.12816],
       [-0.03911,  0.80449,  0.8507 , -0.87633, -1.43986],
       [ 0.50277,  2.04823,  2.10033,  1.33874,  1.87373],
       [ 0.49486,  2.12479,  0.87403,  0.15572,  1.92681],
       [ 1.33346,  0.5233 ,  0.45132,  1.8323 , -0.42377],
       [-1.04615, -1.17159, -1.03068, -2.24563, -0.39959],
       [-0.33494,  0.28132,  0.33921,  0.32351,  0.00905],
       [-0.77962,  0.09586, -0.16656,  0.76871, -0.85234],
       [ 2.55945,  2.35798,  0.43453, -0.20286,  2.23579],
       [ 1.22466, -0.29032,  2.16176, -0.21328,  0.2569 ],
       [ 0.67837, -0.69098, -0.65183, -0.95816, -0.63872

In [8]:
m.Gamma.round(5) # coefficient matrix

array([[-1.02889, -0.90387, -0.26679,  1.21359,  1.62937],
       [ 1.35627,  0.07374,  0.52962,  0.38665,  1.47475],
       [ 0.73702, -0.51402, -1.98329,  0.37549, -0.84382],
       [-0.69373, -0.11328,  1.01238,  1.35226, -0.85822],
       [ 0.90025, -0.19017,  0.16559,  0.54973, -0.08027],
       [-1.31956, -1.33835, -0.70499, -1.92027, -0.08498],
       [ 0.58362, -1.09499, -0.07234, -1.01431,  0.16697],
       [-0.25606,  0.60668,  1.27097, -0.75645, -1.14898],
       [ 0.60632,  1.99315,  1.92343,  1.23539,  1.67268],
       [ 0.7014 ,  2.26436,  0.95187,  0.28101,  1.90697],
       [ 1.08116,  0.36784,  0.74169,  1.86902, -0.23682],
       [-0.97792, -1.0523 , -1.36444, -2.37643, -0.56168],
       [-0.3961 ,  0.15647,  0.82724,  0.68465,  0.34811],
       [-0.61545,  0.30109, -0.61841,  0.52156, -0.93745],
       [ 2.48687,  2.13161,  0.6652 , -0.1914 ,  2.05732],
       [ 1.30188, -0.12781,  1.86845, -0.22011,  0.30709],
       [ 0.3988 , -0.74639, -0.35113, -0.75602, -0.44931

In [10]:
print(f"True (sigma, rho) = ({params['sigma']}, {rho})\nEstimated (sigma, rho) = ({m.sigma:.3f}, {m.rho:.3f})")

True (sigma, rho) = (1, 0.4)
Estimated (sigma, rho) = (1.038, 0.435)


In [11]:
# transform Sigma
Sigma.round(5)

array([[ 0.28   ,  0.22217, -0.73113, -0.16708, -0.40396],
       [ 0.22217,  0.45187, -0.85959, -0.05453, -0.32333],
       [-0.73113, -0.85959,  3.0233 ,  0.49361,  1.41024],
       [-0.16708, -0.05453,  0.49361,  0.34337,  0.41362],
       [-0.40396, -0.32333,  1.41024,  0.41362,  0.90146]])

In [12]:
# estimated transformed Sigma
m.Sigma.round(5)

array([[ 0.12688,  0.08049, -0.24702, -0.06123, -0.12746],
       [ 0.08049,  0.32211, -0.5141 , -0.02136, -0.11754],
       [-0.24702, -0.5141 ,  1.75585,  0.27775,  0.62338],
       [-0.06123, -0.02136,  0.27775,  0.241  ,  0.21881],
       [-0.12746, -0.11754,  0.62338,  0.21881,  0.43512]])

##### Compare to other estimators

In [13]:
# OLS
lm = LinearRegression(fit_intercept=False).fit(X, Y)
B_ols = np.matrix(lm.coef_.transpose())
# Ridge
ridge = RidgeCV(fit_intercept=False).fit(X, Y)
B_ridge = np.matrix(ridge.coef_.transpose())
# Group Lasso
gl = MultiTaskLassoCV(fit_intercept=False).fit(X, Y)
B_gl = np.matrix(gl.coef_.transpose())

In [15]:
# Model Error
print(
f"""
Model Error:
============
MrRCE: {model_error(B, m.Gamma, Sigma_X):.3f}
OLS: {model_error(B, B_ols, Sigma_X):.3f}
GL: {model_error(B, B_gl, Sigma_X):.3f}
Ridge: {model_error(B, B_ridge, Sigma_X):.3f}
"""
)


Model Error:
MrRCE: 1.802
OLS: 3.299
GL: 2.640
Ridge: 2.759



##### Fit with no CV

In [21]:
m = MrRCE(
    max_iter=15,
    tol_glasso=.01,
    tol=1e-3, 
    glasso_cv=False, 
    lam=.15,
    verbose=True
) # init MrRCE

In [22]:
m.fit(X, Y) # fit MrRCE

INFO:root:iter 1, loss 16.635404
INFO:root:iter 2, loss 16.455052
INFO:root:iter 3, loss 16.420933
INFO:root:iter 4, loss 16.411809
INFO:root:iter 5, loss 16.408841
INFO:root:iter 6, loss 16.407830


In [23]:
m.Gamma.round(5) # coefficient matrix

array([[-1.0139 , -0.90582, -0.22931,  1.19986,  1.64887],
       [ 1.3208 ,  0.06838,  0.54277,  0.42599,  1.47998],
       [ 0.75434, -0.4997 , -2.06707,  0.34796, -0.90932],
       [-0.67974, -0.14066,  1.12029,  1.34652, -0.74935],
       [ 0.88775, -0.16864,  0.09862,  0.53726, -0.14598],
       [-1.31383, -1.34721, -0.67829, -1.90229, -0.07604],
       [ 0.55533, -1.06745, -0.1049 , -1.00327,  0.12742],
       [-0.24322,  0.58774,  1.30768, -0.7581 , -1.078  ],
       [ 0.60838,  1.98611,  1.93868,  1.22798,  1.67342],
       [ 0.70883,  2.25516,  0.92604,  0.29409,  1.85053],
       [ 1.06419,  0.39458,  0.72112,  1.84236, -0.20952],
       [-0.97247, -1.03834, -1.39022, -2.36542, -0.58634],
       [-0.39264,  0.13001,  0.89952,  0.686  ,  0.39972],
       [-0.59106,  0.30556, -0.61749,  0.49697, -0.89062],
       [ 2.46223,  2.13282,  0.59832, -0.16854,  1.97152],
       [ 1.28698, -0.12796,  1.93031, -0.21601,  0.38485],
       [ 0.37843, -0.73497, -0.42137, -0.74448, -0.53785

In [24]:
# Model Error
print(
f"""
Model Error:
============
MrRCE (no GLASSO cv): {model_error(B, m.Gamma, Sigma_X):.3f}
OLS: {model_error(B, B_ols, Sigma_X):.3f}
GL: {model_error(B, B_gl, Sigma_X):.3f}
Ridge: {model_error(B, B_ridge, Sigma_X):.3f}
"""
)


Model Error:
MrRCE (no GLASSO cv): 1.931
OLS: 3.299
GL: 2.640
Ridge: 2.759

